#Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#Importing Groq and open AI Keys

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai-key')
os.environ["GROQ_API_KEY"] = userdata.get('groqai-key')


#Importing and Installing OpenAi and GroqAI libraries

In [ ]:
!pip install -qqq openai
!pip install -qqq groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.5 MB/s eta 0:00:00


In [ ]:
from groq import Groq
from openai import OpenAI
import pandas as pd



In [ ]:
client_groq = Groq()
client_openai = OpenAI()

## Define your Groq API call function

In [28]:
def get_completion_groq(prompt, model="llama-3.1-70b-versatile"):
    messages = [{"role": "user", "content": prompt}]
    response = client_groq.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

##Define your OpenAI API call function

In [29]:

def get_completion_openai(prompt, model="gpt-3.5-turbo-0125"):
    messages = [{"role": "user", "content": prompt}]
    response = client_openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Function to profile a user based on their tweets (open AI)

In [30]:
def profile_user_openai(batch_tweets):
    prompt = (
        "Based on the following tweets, identify the user's personal traits, hobbies, likes, and dislikes:\n\n"
        "Tweets:\n"
        + '\n'.join(batch_tweets)
        + "\n\nPersonal Traits:\n"
        "Hobbies:\n"
        "Likes:\n"
        "Dislikes:\n"
    )
    return get_completion_openai(prompt)


## Function to profile a user based on their tweets (Groq AI)

In [31]:
def profile_user_groqai(batch_tweets):
    prompt = (
        "Based on the following tweets, identify the user's personal traits, hobbies, likes, and dislikes:\n\n"
        "Tweets:\n"
        + '\n'.join(batch_tweets)
        + "\n\nPersonal Traits:\n"
        "Hobbies:\n"
        "Likes:\n"
        "Dislikes:\n"
    )
    return get_completion_groq(prompt)

# Function to load and process CSV file

In [32]:
def load_and_process_csv(file_path):
    df = pd.read_csv(file_path)
    print("Available columns:", df.columns)

    if 'tweets' not in df.columns:
        raise KeyError("The 'tweets' column is not found in the CSV file.")

    tweets = df['tweets'].tolist()
    return tweets

## Function to handle batching and combining profiling results using  profile_user_openai function

In [33]:
def batch_process_tweets_openai(tweets, batch_size=200):
    profiles = {
        "Personal Traits": [],
        "Hobbies": [],
        "Likes": [],
        "Dislikes": []
    }
    for i in range(0, len(tweets), batch_size):
        batch = tweets[i:i + batch_size]
        user_profile = profile_user_openai(batch)

        # Extract and accumulate results with checks
        if "Personal Traits:" in user_profile and "Hobbies:" in user_profile:
            personal_traits = user_profile.split("Personal Traits:")[1].split("Hobbies:")[0].strip()
        else:
            personal_traits = ""
        profiles["Personal Traits"].append(personal_traits)

        if "Hobbies:" in user_profile and "Likes:" in user_profile:
            hobbies = user_profile.split("Hobbies:")[1].split("Likes:")[0].strip()
        else:
            hobbies = ""
        profiles["Hobbies"].append(hobbies)

        if "Likes:" in user_profile and "Dislikes:" in user_profile:
            likes = user_profile.split("Likes:")[1].split("Dislikes:")[0].strip()
        else:
            likes = ""
        profiles["Likes"].append(likes)

        if "Dislikes:" in user_profile:
            dislikes = user_profile.split("Dislikes:")[1].strip()
        else:
            dislikes = ""
        profiles["Dislikes"].append(dislikes)

    return profiles


## Function to handle batching and combining profiling results using  profile_user_groqai function

In [34]:
def batch_process_tweets_groq(tweets, batch_size=200):
    profiles = {
        "Personal Traits": [],
        "Hobbies": [],
        "Likes": [],
        "Dislikes": []
    }
    for i in range(0, len(tweets), batch_size):
        batch = tweets[i:i + batch_size]
        user_profile = profile_user_groqai(batch)

        # Extract and accumulate results with checks
        if "Personal Traits:" in user_profile and "Hobbies:" in user_profile:
            personal_traits = user_profile.split("Personal Traits:")[1].split("Hobbies:")[0].strip()
        else:
            personal_traits = ""
        profiles["Personal Traits"].append(personal_traits)

        if "Hobbies:" in user_profile and "Likes:" in user_profile:
            hobbies = user_profile.split("Hobbies:")[1].split("Likes:")[0].strip()
        else:
            hobbies = ""
        profiles["Hobbies"].append(hobbies)

        if "Likes:" in user_profile and "Dislikes:" in user_profile:
            likes = user_profile.split("Likes:")[1].split("Dislikes:")[0].strip()
        else:
            likes = ""
        profiles["Likes"].append(likes)

        if "Dislikes:" in user_profile:
            dislikes = user_profile.split("Dislikes:")[1].strip()
        else:
            dislikes = ""
        profiles["Dislikes"].append(dislikes)

    return profiles


##Function to Summarize the combined profling result for each batch using openai

In [35]:
def summarize_user_profile_openai(user_profile):
    profile_text = f"""
    Personal Traits:
    {''.join(user_profile['Personal Traits'])}

    Hobbies:
    {''.join(user_profile['Hobbies'])}

    Likes:
    {''.join(user_profile['Likes'])}

    Dislikes:
    {''.join(user_profile['Dislikes'])}
    Please analyze the above information and provide a comprehensive summary highlighting the user's main personal traits, hobbies, likes, dislikes, and any other notable characteristics or patterns. Ensure the summary is concise and covers all key aspects of the user's profile.
    """
    response = get_completion_openai(profile_text)
    return response


##Function to Summarize the combined profling result for each batch using groqai

In [75]:
def summarize_user_profile_groq(user_profile):
    profile_text = f"""
    Personal Traits:
    {', '.join(user_profile['Personal Traits'])}

    Hobbies:
    {', '.join(user_profile['Hobbies'])}

    Likes:
    {', '.join(user_profile['Likes'])}

    Dislikes:
    {', '.join(user_profile['Dislikes'])}

    Please analyze the above information and provide a comprehensive summary highlighting the user's main personal traits, hobbies, likes, dislikes, and any other notable characteristics or patterns. Ensure the summary is concise and covers all key aspects of the user's profile.
    """
    response = get_completion_groq(profile_text)
    return response


# Define the path to  the CSV file

In [68]:

csv_file_path = '/content/justinbieber.csv'
csv_file_path1 = '/content/Oprah.csv'
csv_file_path2 = '/content/JeffBezos.csv'
csv_file_path3 = '/content/alexwaarren.csv'
csv_file_path4 = '/content/jakepaul.csv'
csv_file_path5 = '/content/zachking.csv'
csv_file_path6 = '/content/elon_musk_tweets.csv'
csv_file_path7 = '/content/trump_tweets.csv'


## Batch process the tweets

In [69]:
user_tweets = load_and_process_csv(csv_file_path)
user_tweets1 = load_and_process_csv(csv_file_path1)
user_tweets2 = load_and_process_csv(csv_file_path2)
user_tweets3 = load_and_process_csv(csv_file_path3)
user_tweets4 = load_and_process_csv(csv_file_path4)
user_tweets5 = load_and_process_csv(csv_file_path5)
user_tweets6 = load_and_process_csv(csv_file_path6)
user_tweets7 = load_and_process_csv(csv_file_path7)

Available columns: Index(['tweets'], dtype='object')
Available columns: Index(['tweets'], dtype='object')
Available columns: Index(['tweets'], dtype='object')
Available columns: Index(['tweets'], dtype='object')
Available columns: Index(['tweets'], dtype='object')
Available columns: Index(['tweets'], dtype='object')
Available columns: Index(['tweets'], dtype='object')
Available columns: Index(['tweets'], dtype='object')


#Get the profile of the Justin Bieber based on his tweets

**Justin Drew Bieber is a Canadian singer. Regarded as a pop icon, he is recognized for his multi-genre musical performances**

###OpenAI Response for all the batches

In [39]:
profile_summary_openai = batch_process_tweets_openai(user_tweets)
profile_summary_openai

{'Personal Traits': ['Family-oriented, loving, supportive, charitable, passionate, dedicated, grateful, reflective, creative, musical',
  'Justin Bieber appears to be grateful, caring, and passionate about his music and fans. He also seems to be determined and willing to speak up for what he believes in.',
  'Grateful, caring, appreciative, dedicated, honest, excited, fun-loving',
  'Grateful, supportive, loving, creative',
  'Compassionate, supportive, grateful, positive, caring, empathetic, determined, passionate, outspoken, reflective, humble.',
  'Supportive, grateful, caring, determined, positive, humble, loyal',
  'Friendly, grateful, supportive, hardworking, happy, positive',
  'Grateful, caring, appreciative, supportive, friendly, outgoing, hardworking, dedicated, emotional',
  'grateful, dedicated, hardworking, caring, humorous, appreciative, loyal',
  'Grateful, appreciative, hardworking, dedicated, loyal, caring, positive, inspirational, honest, determined, friendly, fun-lov

#### Summarize the response of all batches  

In [47]:
opeani_twitter_profiling = summarize_user_profile_openai(profile_summary_openai)
opeani_twitter_profiling





'Justin Bieber is a family-oriented, grateful, and hardworking individual who is passionate about music and dedicated to his fans. He enjoys performing, creating music, collaborating with other artists, and supporting charitable causes. Justin dislikes negativity, rumors, and being misunderstood, but remains positive and focused on spreading love and kindness.'

###Groq Response

In [73]:
profile_summary_groq = batch_process_tweets_groq(user_tweets)
profile_summary_groq

{'Personal Traits': ['**\n\n* Romantic: The user frequently expresses love and appreciation for their partner, Hailey Bieber.\n* Supportive: The user often shows support for their friends, family, and colleagues, congratulating them on their achievements and expressing gratitude for their presence in their life.\n* Passionate: The user is passionate about their music, hockey, and social causes, frequently expressing excitement and enthusiasm for upcoming events and projects.\n* Grateful: The user often expresses gratitude for their blessings, opportunities, and relationships.\n* Humble: The user acknowledges their privilege and uses their platform to raise awareness for social causes and support others.\n\n**',
  '**\n\n* Creative: The user is a musician and artist, and their tweets showcase their creative process and passion for music.\n* Empathetic: The user shows concern for others, particularly in times of crisis, and uses their platform to raise awareness and support for social ca

#### Pass the profile to the Groq to summarize it

In [76]:
groq_twitter_profiling = summarize_user_profile_groq(profile_summary_groq)
groq_twitter_profiling

"Based on the provided information, here is a comprehensive summary of the user's profile:\n\n**Personal Traits:**\n\n* Romantic: The user frequently expresses love and appreciation for their partner, Hailey Bieber.\n* Supportive: The user often shows support for their friends, family, and colleagues, congratulating them on their achievements and expressing gratitude for their presence in their life.\n* Passionate: The user is passionate about their music, hockey, and social causes, frequently expressing excitement and enthusiasm for upcoming events and projects.\n* Grateful: The user often expresses gratitude for their blessings, opportunities, and relationships.\n* Humble: The user acknowledges their privilege and uses their platform to raise awareness for social causes and support others.\n\n**Hobbies:**\n\n* Music: The user is a musician and frequently tweets about their music, upcoming performances, and collaborations.\n* Hockey: The user is a hockey fan and occasionally tweets ab

##Get the profile of the Oprah Winfrey based on her tweets
**Oprah Gail Winfrey known mononymously as Oprah, is an American talk show host, television producer, actress, author, and media proprietor.**

###OpenAI Response for all the batches

In [53]:
profile_summary_openai_oprah = batch_process_tweets_openai(user_tweets1)
profile_summary_openai_oprah


{'Personal Traits': ['Empathetic, compassionate, supportive, resilient, determined, strong-willed, caring, nurturing',
  'Empathetic, compassionate, supportive, influential, determined, resilient, strong-willed, passionate, caring, dedicated, insightful, introspective, reflective, optimistic, inspiring.',
  'Empathetic, compassionate, socially conscious, influential, caring, supportive, optimistic, resilient, determined, strong-willed, open-minded, inclusive, nurturing',
  'Compassionate, empathetic, spiritual, influential, supportive, enthusiastic, positive, grateful, open-minded',
  'Compassionate, empathetic, supportive, passionate, dedicated, influential, caring, generous',
  'Empathetic, compassionate, insightful, spiritual, influential, caring, supportive',
  'Supportive, positive, engaged, empathetic, spiritual',
  'Supportive, empathetic, expressive, passionate, spiritual',
  'Spiritual, compassionate, family-oriented, reflective, grateful',
  'Supportive, grateful, engaged, em

#### Summarize the response of all batches

In [54]:
opeani_twitter_profiling_oprah = summarize_user_profile_openai(profile_summary_openai_oprah)
opeani_twitter_profiling_oprah

'The user appears to be a compassionate, empathetic, and spiritual individual who enjoys engaging in meaningful conversations, supporting social causes, and promoting positivity. They have a strong interest in empowering others, celebrating achievements, and fostering personal growth. The user dislikes negativity, lack of empathy, and injustice, and values authenticity, self-improvement, and connection with others.'

###Groq Response##

In [77]:
profile_summary_groq_oprah = batch_process_tweets_groq(user_tweets1)
profile_summary_groq_oprah

{'Personal Traits': ['**\n\n* Empathetic: The user shows empathy towards others, especially those who have experienced trauma or hardship.\n* Supportive: The user is supportive of others, offering words of encouragement and praise.\n* Passionate: The user is passionate about social justice, equality, and mental health awareness.\n* Inspirational: The user aims to inspire and motivate others through their words and actions.\n* Grateful: The user expresses gratitude towards others and acknowledges the importance of appreciation.\n\n**',
  '**\n\n* Empathetic and compassionate: The user frequently expresses gratitude, support, and admiration for others, indicating a caring and empathetic personality.\n* Inspirational and motivational: The user often shares motivational quotes, stories, and messages, suggesting a desire to inspire and uplift others.\n* Open-minded and curious: The user engages in conversations about various topics, including mental health, social justice, and personal grow

In [78]:
groq_twitter_profiling_oprah = summarize_user_profile_groq(profile_summary_groq_oprah)
groq_twitter_profiling_oprah

"Based on the provided information, here is a comprehensive summary of the user's profile:\n\n**Personal Traits:**\n\n* Empathetic and compassionate, with a strong sense of social justice and equality\n* Supportive and encouraging, with a desire to help others and make a positive impact\n* Passionate about personal growth and self-improvement, with a focus on spirituality and mindfulness\n* Authentic and genuine, with a willingness to be vulnerable and share personal thoughts and feelings\n* Grateful and appreciative, with a focus on expressing thanks and gratitude to others\n\n**Hobbies:**\n\n* Reading, particularly self-help and spiritual books\n* Watching TV shows and movies, especially those that promote social justice and equality\n* Traveling and exploring new places, with a desire to experience different cultures and meet new people\n* Meditating and practicing mindfulness, with a focus on personal growth and self-awareness\n* Spending time in nature, with a appreciation for the

##Get the profile of the Jeff Bezos based on the tweets
**Jeffrey Preston Bezos is an American business magnate best known as the founder, executive chairman, and former president and CEO of Amazon.**

###OpenAI Response###

In [55]:
profile_summary_openai_jeff = batch_process_tweets_openai(user_tweets2)
profile_summary_openai_jeff

{'Personal Traits': ['Philanthropic, innovative, hardworking, family-oriented, patriotic, appreciative, adventurous, curious, supportive, proud, grateful',
  'Intelligent, ambitious, hardworking, caring, family-oriented'],
 'Hobbies': ['Space exploration, philanthropy, investing, reading, watching TV shows and movies, traveling, attending events and conferences',
  'Rocketry, space exploration, robotics, reading, watching movies'],
 'Likes': ['Family, technology, space, philanthropy, innovation, hard work, success, learning, supporting others, honoring achievements, spending time with loved ones',
  'Amazon, Blue Origin, technology, innovation, family, friends, success, achievements, space travel'],
 'Dislikes': ['Injustice, inefficiency, lack of progress, disrespect, negativity, lack of effort, lack of appreciation, lack of support, lack of innovation',
  'Inefficiency, failure, negativity, lack of progress']}

In [56]:
opeani_twitter_profiling_jeff = summarize_user_profile_openai(profile_summary_openai_jeff)
opeani_twitter_profiling_jeff

'This user is a philanthropic, innovative, and hardworking individual who is family-oriented and patriotic. They have a strong interest in space exploration, robotics, and investing, and enjoy reading, watching movies, and attending events. They value technology, innovation, and supporting others, while disliking injustice, inefficiency, and negativity. Their profile reflects a curious, adventurous, and supportive nature with a focus on personal growth and achievement.'

###Groq Response##

In [79]:
profile_summary_groq_jeff = batch_process_tweets_groq(user_tweets2)
profile_summary_groq_jeff

{'Personal Traits': ["**\n\n* Philanthropic: The user is involved in various charitable initiatives and expresses gratitude towards others who are making a positive impact.\n* Supportive: The user shows appreciation and support for their family, friends, and colleagues.\n* Humble: The user acknowledges their own mistakes and failures, and expresses admiration for others' achievements.\n* Curious: The user is interested in learning and exploring new ideas, as evident from their engagement with various topics and people.\n* Passionate: The user is enthusiastic about their work, interests, and values.\n\n**",
  '**\n\n* Adventurous: The user is involved in space exploration and rocket launches, indicating a sense of adventure and willingness to take risks.\n* Curious: The user is interested in learning and exploring new ideas, as evident from their engagement with various topics, including science, technology, and innovation.\n* Appreciative: The user frequently expresses gratitude and ap

In [80]:
groq_twitter_profiling_jeff = summarize_user_profile_groq(profile_summary_groq_jeff)
groq_twitter_profiling_jeff

"**Comprehensive Summary**\n\nThe user is a philanthropic, supportive, and humble individual who is passionate about making a positive impact. They are curious, enthusiastic, and adventurous, with a strong interest in space exploration, technology, and innovation. The user values their family, friends, and colleagues, and is grateful for the work of others who are making a difference.\n\n**Personal Traits:**\n\n* Philanthropic: Involved in charitable initiatives and expresses gratitude towards others making a positive impact.\n* Supportive: Appreciative of family, friends, and colleagues, and celebrates their achievements.\n* Humble: Acknowledges mistakes and failures, and admires others' achievements.\n* Curious: Interested in learning and exploring new ideas, topics, and people.\n* Passionate: Enthusiastic about work, interests, and values.\n* Adventurous: Involved in space exploration and rocket launches, indicating a sense of adventure and willingness to take risks.\n\n**Hobbies:**

##Get the profile of the Alex Waarren based on the tweets
**Alexander Warren Hughes is an American singer-songwriter and comedic video creator on YouTube. He is also popular on TikTok and Instagram.**

In [57]:
profile_summary_openai_warren = batch_process_tweets_openai(user_tweets3)
profile_summary_openai_warren

{'Personal Traits': ['Funny, caring, creative, emotional, reflective, curious, adventurous',
  '',
  'Funny, outgoing, caring, creative, spontaneous, curious, appreciative, supportive, energetic, adventurous',
  'Funny, caring, hardworking, dedicated, loyal, easily distracted, easily stressed, loves animals, sentimental, family-oriented',
  'Friendly, caring, grateful, easily distracted, humorous, easily overwhelmed, supportive, curious, adventurous, creative, hardworking, dedicated, ambitious, social, easily excitable',
  'Funny, caring, creative, adventurous, social, easily distracted, impulsive',
  'Friendly, caring, grateful, humorous, energetic, creative, adventurous',
  'Funny, caring, supportive, adventurous, impulsive',
  'Active on social media, dedicated to creating content, appreciative of fans, caring towards friends, humorous, expressive, open about personal life',
  'Social, outgoing, adventurous, humorous, caring, creative'],
 'Hobbies': ['Making music, vlogging, podcast

In [58]:
opeani_twitter_profiling_warren = summarize_user_profile_openai(profile_summary_openai_warren)
opeani_twitter_profiling_warren

'This user is a funny, caring, and creative individual who enjoys vlogging, creating music, and interacting with fans. They have a strong passion for supporting mental health awareness and charity work. However, they dislike drama, negative comments, and feeling overwhelmed. Overall, they are outgoing, adventurous, and dedicated to creating engaging content for their followers.'

###Groq Response###

In [81]:
profile_summary_groq_warren = batch_process_tweets_groq(user_tweets3)
profile_summary_groq_warren

{'Personal Traits': ['**\n\n* Emotional and expressive: The user frequently shares their feelings and emotions on Twitter, often using phrases like "I love u" and "I\'m so happy."\n* Sarcastic and humorous: The user often uses humor and sarcasm in their tweets, often making light of serious situations.\n* Creative: The user is a musician and songwriter, and frequently shares updates about their music and creative projects.\n* Open and honest: The user is willing to share personal details and thoughts on Twitter, including their relationships and struggles.\n* Playful and goofy: The user often uses playful language and emojis, and engages in lighthearted banter with their followers.\n\n**',
  '**\n\n* Emotional and expressive: The user frequently shares their emotions, thoughts, and feelings on Twitter.\n* Humorous and sarcastic: The user often uses humor and sarcasm to express themselves.\n* Creative and artistic: The user is involved in music, video production, and other creative purs

In [82]:
groq_twitter_profiling_warren = summarize_user_profile_groq(profile_summary_groq_warren)
groq_twitter_profiling_warren

'**Comprehensive Summary**\n\nThe user is a young adult, likely in their late teens or early twenties, who is active on various social media platforms, including Twitter, YouTube, and TikTok. They are a content creator, musician, and songwriter who enjoys making videos, music, and vlogs.\n\n**Personal Traits:**\n\n* Emotional and expressive, often sharing their feelings and emotions on social media\n* Humorous and sarcastic, frequently using humor to cope with serious situations\n* Creative and artistic, with a passion for music, video production, and other creative pursuits\n* Open and honest, willing to share personal details and thoughts on social media\n* Confident and self-assured, but also vulnerable and empathetic\n* Playful and goofy, often using playful language and emojis in their online interactions\n\n**Hobbies:**\n\n* Creating content for YouTube and other social media platforms\n* Music and songwriting\n* Video production and editing\n* Gaming and streaming on Twitch\n* T

##Get the profile of the Jake Joseph Paul based on the tweets
**Jake Joseph Paul  is an American YouTuber, actor, and professional boxer.**

In [62]:
profile_summary_openai_jake = batch_process_tweets_openai(user_tweets4)
profile_summary_openai_jake

{'Personal Traits': ['Confident, assertive, passionate, determined, competitive, outspoken, generous, grateful, respectful, supportive, patriotic',
  'Confident, outspoken, competitive, determined, passionate',
  'Confident, assertive, outspoken, determined, ambitious',
  'Confident, competitive, outspoken, determined, generous, ambitious, passionate',
  'Confident, outspoken, competitive, humorous, charitable',
  'Confident, ambitious, outspoken, competitive, determined, strategic',
  'Confident, competitive, outspoken, determined, ambitious, social, humorous, passionate',
  'Confident, ambitious, outspoken, humorous, caring, determined, impulsive, rebellious',
  'Confident, outspoken, humorous, caring, supportive, loyal',
  'Confident, outgoing, creative, determined, caring, loyal',
  'Emotional, reflective, caring, determined, hardworking, loyal, confident, humorous',
  'Emotional, loyal, determined, confident, caring, humorous, hardworking',
  'Confident, outgoing, determined, cari

In [63]:
opeani_twitter_profiling_jake = summarize_user_profile_openai(profile_summary_openai_jake)
opeani_twitter_profiling_jake

'The user appears to be a confident, outgoing, and humorous individual who is passionate about sports, particularly boxing and wrestling. They enjoy creating content, interacting with fans on social media, and engaging in competitive activities. The user values positivity, family, and supporting others, while disliking negativity, disrespect, and being underestimated. Overall, they have a fun-loving and energetic personality with a strong sense of loyalty and determination.'

###Groq Response###

In [85]:
profile_summary_groq_jake = batch_process_tweets_groq(user_tweets4)
profile_summary_groq_jake

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01j4k9t0rdfg8vn0grca9cj5k6` on : Limit 1000000, Used 996594, Requested 4192. Please try again in 1m7.880799999s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [84]:
groq_twitter_profiling_jake = summarize_user_profile_groq(profile_summary_groq_jake)
groq_twitter_profiling_jake

NameError: name 'profile_summary_groq_jake' is not defined

##Get the profile of the Zach King based on the tweets
**Zachary Michael "Zach" King  is an American internet personality based in Los Angeles.He is most known for his "magic vines," which are six-second videos digitally edited to look as if he is doing magic.**

In [64]:
profile_summary_openai_zach = batch_process_tweets_openai(user_tweets5)
profile_summary_openai_zach

{'Personal Traits': ['Creative, adventurous, generous, competitive, tech-savvy, positive, grateful, collaborative',
  'Creative, adventurous, family-oriented, tech-savvy, friendly, supportive',
  'Generous, creative, hardworking, proud, curious, tech-savvy, family-oriented',
  'Creative, adventurous, friendly, humorous',
  'Adventurous, creative, friendly, grateful, humorous, tech-savvy',
  'Friendly, grateful, humorous, adventurous, creative',
  'Grateful, hardworking, creative, adventurous, humorous, caring',
  'Creative, adventurous, friendly, humorous, grateful',
  'Friendly, grateful, humorous, adventurous, creative',
  'Creative, adventurous, social, tech-savvy, humorous',
  'Creative, adventurous, social, humorous, confident, grateful, appreciative, friendly, outgoing, ambitious',
  'Creative, humorous, adventurous, tech-savvy, friendly, appreciative',
  'Adventurous, social, friendly, creative, hardworking, dedicated, tech-savvy',
  'Creative, tech-savvy, friendly, responsive, 

In [65]:
opeani_twitter_profiling_zach = summarize_user_profile_openai(profile_summary_openai_zach)
opeani_twitter_profiling_zach

'The user is a creative and tech-savvy individual who enjoys filmmaking, editing videos, and collaborating with others. They are adventurous, friendly, and grateful, with a passion for exploring new technologies and creating unique content. They enjoy interacting with fans, traveling, and participating in charity work. However, they dislike technical difficulties, waiting in line, and negative feedback.'

##Get the profile of the Elon Musk based on the tweets

**Elon Reeve Musk FRS  is a businessman and investor known for his key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.**

In [66]:
profile_summary_openai_elon = batch_process_tweets_openai(user_tweets6)
profile_summary_openai_elon

{'Personal Traits': ['Intelligent, innovative, ambitious, opinionated, outspoken, humorous, sarcastic, determined, visionary',
  'Intelligent, opinionated, humorous, sarcastic, confident, ambitious, hardworking, detail-oriented, innovative, controversial, influential.',
  'Intelligent, innovative, detail-oriented, opinionated, determined, passionate, critical, humorous, sarcastic, creative',
  'Intelligent, analytical, opinionated, determined, creative, humorous, caring, concerned about global issues',
  'Intelligent, opinionated, determined, innovative, sarcastic, humorous, politically aware',
  'Independent, opinionated, sarcastic, humorous, tech-savvy, determined, detail-oriented, creative, open-minded, controversial',
  'Confident, opinionated, detail-oriented, tech-savvy, humorous, analytical',
  'Intelligent, opinionated, determined, focused, detail-oriented, analytical, critical, sarcastic, humorous, confident, ambitious.',
  'Intelligent, opinionated, determined, passionate, ou

In [67]:
opeani_twitter_profiling_elon = summarize_user_profile_openai(profile_summary_openai_elon)
opeani_twitter_profiling_elon

'The user is intelligent, opinionated, and tech-savvy, with a strong interest in technology, space exploration, and social media. They enjoy discussing current events, engaging in debates, and following advancements in AI. The user appreciates innovation, transparency, and free speech, while disliking misinformation, censorship, and media bias. Their hobbies include reading, coding, and advocating for sustainable energy solutions.'

##Get the profile of the Trump based on the tweets
**Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021.**

In [70]:
profile_summary_openai_trump = batch_process_tweets_openai(user_tweets7)
profile_summary_openai_trump

{'Personal Traits': ['Patriotic, confident, outspoken, determined, focused, loyal, proud, empathetic, sympathetic, compassionate',
  'Confident, assertive, proud, determined, patriotic, loyal, outspoken, controversial',
  'Supportive, confident, assertive, patriotic, determined, outspoken, controversial, competitive',
  'Strong opinions, confident, assertive, outspoken, determined, focused, patriotic',
  'Strong opinions, confident, assertive, focused on politics, patriotic',
  'Patriotic, confident, outspoken, determined, competitive',
  'Strong opinions, confident, assertive, determined, patriotic, outspoken, controversial',
  'Strong-willed, determined, confident, outspoken, controversial, conspiracy theorist',
  'Strong opinions, determined, outspoken, confident, persistent',
  'The user appears to be very opinionated, outspoken, and confident in their beliefs. They also seem to be very politically active and engaged.'],
 'Hobbies': ['Golfing, watching sports, reading, writing, tra

In [72]:
opeani_twitter_profiling_trump = summarize_user_profile_openai(profile_summary_openai_trump)
opeani_twitter_profiling_trump

'The user is patriotic, confident, and outspoken, with a strong focus on politics and conservative values. They enjoy activities like golfing and attending rallies, and appreciate support from others and endorsements. However, they strongly dislike what they perceive as corruption, voter fraud, and incompetence in the political system, as well as certain politicians and media outlets.'

##Conclusion##

In this project, I used both  OpenAI(gpt-3.5-turbo-0125) and Lama(llama-3.1-70b-versatile) to profile **eight users** based on their tweets, aiming to infer personal traits such as likes, dislikes, hobbies etc. The generated profiles were compared with publicly available information, revealing that while the LLM accurately identified several traits, there were notable gaps due to the limited dataset. These gaps resulted from the insufficient number and variety of tweets, leading to incomplete profiles. The accuracy and detail of the profiling were found to be heavily dependent on the diversity and richness of the tweet data. Users with more varied tweets had more accurate profiles, whereas those with fewer tweets had less detailed profiles. This underscores the importance of having a comprehensive dataset for effective social media analysis. Future efforts should focus on expanding the dataset to improve the accuracy and completeness of user profiling.